In [64]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
import torch.optim as optim

In [65]:
df = pd.read_csv("../../Datensatz/data.csv")

In [66]:
# Wandle die Spalten "age" und "emotion" in NumPy-Arrays um
X = torch.from_numpy(df[["age", "emotions"]].values).float()
# Wandle das Zielkolumnen-Array in einen PyTorch-Tensor um
y = torch.from_numpy(df[["product"]].values).long()

In [67]:
# Erstelle die PyTorch-Datensätze
dataset = torch.utils.data.TensorDataset(X, y)

# Teile den Datensatz in Trainings- und Validierungssätze auf
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Erstelle die Dataloader für das Training und die Validierung
train_dataloader = DataLoader(train_dataset, batch_size=32)
test_dataloader = DataLoader(val_dataset, batch_size=1)


In [68]:

for X, y in test_dataloader:
    print(f"Shape of X [age,emotions]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [age,emotions]: torch.Size([1, 2])
Shape of y: torch.Size([1, 1]) torch.int64


In [69]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 26)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=26, bias=True)
  )
)


In [70]:
criterion = nn.CrossEntropyLoss()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [71]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, torch.max(y, 1)[1])

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [72]:
for batch, (X, y) in enumerate(train_dataloader):
    #print(f"X={X}\ny={y}\n\n")
    X, y = X.to(device), y.to(device)
    print(f"X={X}\ny={y}\nx_shape={X.shape}\n y_shape={y.shape}\nmodel(X)={model(X).shape}\n\n")


X=tensor([[1., 2.],
        [3., 4.],
        [3., 5.],
        [1., 4.],
        [1., 5.],
        [2., 1.],
        [3., 2.],
        [2., 7.],
        [2., 1.],
        [2., 5.],
        [2., 1.],
        [2., 4.],
        [2., 3.],
        [3., 5.],
        [1., 1.],
        [3., 3.],
        [2., 7.],
        [3., 4.],
        [2., 5.],
        [1., 2.],
        [2., 5.],
        [2., 1.],
        [3., 2.],
        [3., 2.],
        [1., 4.],
        [3., 2.],
        [2., 3.],
        [2., 3.],
        [2., 1.],
        [2., 4.],
        [2., 3.],
        [3., 7.]])
y=tensor([[19],
        [ 1],
        [10],
        [19],
        [ 1],
        [ 1],
        [25],
        [13],
        [ 7],
        [18],
        [23],
        [13],
        [14],
        [20],
        [ 5],
        [ 7],
        [ 2],
        [14],
        [ 4],
        [19],
        [22],
        [11],
        [10],
        [21],
        [ 8],
        [ 5],
        [20],
        [18],
        [14],
        [16],

In [73]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, torch.max(y, 1)[1]).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [74]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 3.865599  [    0/16000]
loss: 0.008027  [ 3200/16000]
loss: 0.013364  [ 6400/16000]
loss: 0.006933  [ 9600/16000]
loss: 0.001706  [12800/16000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.002512 

Epoch 2
-------------------------------
loss: 0.003560  [    0/16000]
loss: 0.000603  [ 3200/16000]
loss: 0.003196  [ 6400/16000]
loss: 0.002417  [ 9600/16000]
loss: 0.000534  [12800/16000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.001131 

Epoch 3
-------------------------------
loss: 0.001570  [    0/16000]
loss: 0.000262  [ 3200/16000]
loss: 0.001769  [ 6400/16000]
loss: 0.001447  [ 9600/16000]
loss: 0.000287  [12800/16000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.000716 

Epoch 4
-------------------------------
loss: 0.000981  [    0/16000]
loss: 0.000157  [ 3200/16000]
loss: 0.001212  [ 6400/16000]
loss: 0.001028  [ 9600/16000]
loss: 0.000187  [12800/16000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.000519 

Epoch 5
----------------------------

In [75]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [76]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [77]:
X

tensor([[2., 4.],
        [2., 5.],
        [2., 7.],
        [2., 3.],
        [2., 5.],
        [3., 5.],
        [3., 5.],
        [1., 5.],
        [1., 6.],
        [1., 3.],
        [2., 3.],
        [3., 2.],
        [2., 1.],
        [2., 5.],
        [3., 4.],
        [1., 7.],
        [2., 1.],
        [3., 1.],
        [2., 6.],
        [2., 2.],
        [3., 7.],
        [2., 7.],
        [1., 7.],
        [2., 4.],
        [3., 7.],
        [2., 5.],
        [2., 7.],
        [2., 3.],
        [2., 6.],
        [2., 6.],
        [3., 6.],
        [1., 5.]])

In [78]:
val_dataset[0][0]

tensor([1., 2.])

In [81]:
model(X)

tensor([[ 1.8455e+01, -3.8814e-01, -2.0549e+00, -1.0403e+00,  4.7515e-01,
         -6.5865e-01, -1.0767e+00, -8.2211e-01, -1.1761e+00, -1.5299e+00,
         -2.1660e+00, -3.9935e-01, -5.6999e-01, -2.6447e+00,  2.6174e-01,
         -5.4645e-01, -4.3299e-01, -8.4983e-01, -1.0783e-01, -1.4463e+00,
         -4.4743e-01, -5.7467e-02, -1.2836e+00, -1.8059e+00,  4.3580e-01,
          4.6409e-01],
        [ 2.1612e+01, -5.2450e-01, -2.4131e+00, -1.2655e+00,  5.7234e-01,
         -8.0047e-01, -1.2897e+00, -9.7330e-01, -1.3224e+00, -1.7447e+00,
         -2.5168e+00, -4.4692e-01, -6.3025e-01, -3.0855e+00,  2.8688e-01,
         -6.9901e-01, -5.5247e-01, -1.0502e+00, -1.6179e-01, -1.6955e+00,
         -5.4508e-01, -7.9055e-02, -1.4925e+00, -2.1056e+00,  5.2051e-01,
          6.0739e-01],
        [ 2.7980e+01, -7.7729e-01, -3.0957e+00, -1.7196e+00,  7.7144e-01,
         -1.1141e+00, -1.7063e+00, -1.2662e+00, -1.5863e+00, -2.1801e+00,
         -3.2439e+00, -5.2487e-01, -7.3342e-01, -3.9810e+00,  3.58

In [ ]:
classes = ["Girokonto","Gemeinschaftskonto","Kreditkarte","Tagesgeldkonto","Sparplan","Bausparplan",
"Edelmetall Depot","Aktien Depot","Aktiensparplan","ETF Sparplan","Privatkredit","Umschuldung","Immobilienfinanzierung",
"Immobilien","Hebel Zertifikate","Crypto","Lebensversicherung","Rentenversicherung","NFT","Berufsunfähigkeitsversicherung",
"Crypto, Hebel Zertifikate","Immobilien, Bausparvertrag","Gemeinschaftskonto, Tagesgeldkonto","NFT, Crypto","Staatsanleihen","Bausparvertrag, Aktien Sparplan"]

model.eval()
x,y=val_dataset[0][0],val_dataset[0][1]
with torch.no_grad():
    pred = model(X)
    predicted, actual = classes[pred[0].argmax(0)], classes[torch.max(y, 1)[1]]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)